Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-how-to-use-modulestep.png)


# How to create Module, ModuleVersion, and use them in a pipeline with ModuleStep.
In this notebook, we introduce the concept of versioned modules and how to use them in an Azure Machine Learning Pipeline.

The core idea behind introducing Module, ModuleVersion and ModuleStep is to allow the separation between reusable executable components and their actual usage. These reusable software components (such as scripts or executables) can be used in different scenarios and by different users. This follows the same idea of separating software frameworks/libraries and their actual usage in applications. Module and ModuleVersion take the role of the reusable executable components where ModuleStep is there to link them to an actual usage.

A module is an elaborated container of its versions, where each version is the actual computational unit. It is up to users to define the semantics of this hierarchical structure of container and versions. For example, there could be different versions for different use cases, development progress, etc.

Each ModuleVersion may have inputs, outputs and rely on parameters and its environment configuration to operate.

Because Modules can now be separated from execution in a pipeline, there's a need for a mechanism to reconnect these again, and allow using Modules and their versions in a Pipeline. This is done using a new kind of Step called ModuleStep, which allows embedding a Module (and more precisely, a version of it) in a Pipeline.
 
This notebook shows the usage of a module that computes the sum and product of two numbers. As a module can only be used as a step in a pipeline, we define two different versions for it, to be used in two different use cases:

1) As the module powering the initial step of a pipeline, where the step does not receive any input from preceding steps.

2) As a module powering a step in the pipeline that receives inputs from preceding steps.

Once these two versions are defined, we show how to embed them as steps in the pipeline.

## Prerequisites and AML Basics
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration Notebook](https://aka.ms/pl-config) first if you haven't. This sets you up with a working config file that has information on your workspace, subscription id, etc.

### Initialization Steps

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, RunConfiguration
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.pipeline.core import Pipeline, PipelineData, PipelineParameter
from azureml.pipeline.core.graph import InputPortDef, OutputPortDef
from azureml.pipeline.core.module import Module
from azureml.pipeline.steps import ModuleStep

workspace = Workspace.from_config()
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, sep = '\n')

aml_compute_target = "cpu-cluster"
try:
    aml_compute = AmlCompute(workspace, aml_compute_target)
    print("Found existing compute target: {}".format(aml_compute_target))
except:
    print("Creating new compute target: {}".format(aml_compute_target))
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(workspace, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

datastore = Datastore(workspace=workspace, name="workspaceblobstore")

## Create a Module

A Module is a container that manages computational units. Each such computational unit is a version of the module, and is called a ModuleVersion. We start by either creating a module or fetching an existing one by its ID or by its name.

In [ ]:
module = Module.create(workspace, name="AddAndMultiply", description="A module that adds and multiplies")

### Calculation entry ModuleVersion

A ModuleVersion is an actual computational unit. Defining it involves defining its inputs, outputs, the computation and other configuration items. 

Here we define that this version is to be used at the beginning of the pipeline, hence does not have incoming ports, only outgoing.

In [ ]:
out_sum = OutputPortDef(name="out_sum", default_datastore_name=datastore.name, default_datastore_mode="mount", 
                        label="Sum of two numbers")
out_prod = OutputPortDef(name="out_prod", default_datastore_name=datastore.name, default_datastore_mode="mount", 
                         label="Product of two numbers")
entry_version = module.publish_python_script("calculate.py", "initial", 
                                             inputs=[], outputs=[out_sum, out_prod], params = {"initialNum":12},
                                             version="1", source_directory="./calc")

### Calculation middle/end ModuleVersion

Another version of the module performs a computation in the middle or at the end of the pipeline. This version has both outputs and inputs, as it is to be either followed by another computation, or emits its outputs.

In [ ]:
in1_mid = InputPortDef(name="in1", default_datastore_mode="mount", 
                   default_data_reference_name=datastore.name, label="First input number")
in2_mid = InputPortDef(name="in2", default_datastore_mode="mount", 
                   default_data_reference_name=datastore.name, label="Second input number")
out_sum_mid = OutputPortDef(name="out_sum", default_datastore_name=datastore.name, default_datastore_mode="mount",
                            label="Sum of two numbers")
out_prod_mid = OutputPortDef(name="out_prod", default_datastore_name=datastore.name, default_datastore_mode="mount",
                             label="Product of two numbers")
module.publish_python_script(
    "calculate.py", "middle", inputs=[in1_mid, in2_mid], outputs=[out_sum_mid, out_prod_mid], version="2", is_default=True, 
    source_directory="./calc")

## Using a Module in a Pipeline with ModuleStep

### Introduction

Using a Module, and more precisely, a specific version, in a pipeline is done via a specialized kind of step. This step is called ModuleStep. It is used as a step in a pipeline, one that holds enough information that allows pinpointing to a specific ModuleVersion. 

Another responsibility of a ModuleStep is to wire the actual data that is used in the pipeline to the inputs/outputs definitions of the ModuleVersion. This wiring is done by mapping each of the inputs and the outputs definitions to a data element in the pipeline. Defining the wiring is done using a dictionary whose keys are the name of the inputs/outputs, and the mapped value is the data element (e.g., a PipelineData object).

#### Deciding which ModuleVersion to use - resolving

It is up to the ModuleStep to decide which ModuleVersion to use. That decision is based on the parameters given to the ModuleStep, and it follows this process:
1. If a ModuleVersion object was provided, use it.
2. For the given Module object, if a version was provided, use it.
3. The given Module object resolves which is the right version:
  1. If a default ModuleVersion was defined for the Module, use it.
  2. If all the versions of the ModuleVersions in the Module follow semantic versioning, take the one with the highest version.
  3. Take the ModuleVersion with the most recent update.

### First Step and its wires

<p>The first step in a pipeline does not have incoming inputs, but it does have outputs. For that we'd use the ModuleVersion that was designed for this use case.</p>
We start off by preparing the outgoing edges as two PipelineData objects (to be later linked to another step), as well as wiring these to the moduleVersion's outputs by creating a dictionary mapping.

In [ ]:
first_sum = PipelineData("sum_out", datastore=datastore, output_mode="mount",is_directory=False)
first_prod = PipelineData("prod_out", datastore=datastore, output_mode="mount",is_directory=False)
step_output_wiring = {"out_sum":first_sum, "out_prod":first_prod}

#### Initial ModuleStep

<p>In order for the step to know which ModuleVersion to use, we provided the initial ModuleVersion object. We wire the ModuleVersion's outputs with the <i> step_output_wiring</i> map we just created. </p>
The initial ModuleStep uses the ModuleVersion that does not have inputs from the pipeline, however, it still needs to receive two numbers to operate upon. We'll provide these numbers as arguments to the step. The first is provided as a parameter, the other one is hard coded.

In [ ]:
first_num_param = PipelineParameter(name="initialNum", default_value=17)
first_step = ModuleStep(module_version=entry_version,
                 inputs_map={}, outputs_map=step_output_wiring, 
                 runconfig=RunConfiguration(), 
                 compute_target=aml_compute, 
                 arguments = ["--output_sum", first_sum, 
                              "--output_product", first_prod,
                              "--arg_num1", first_num_param, 
                              "--arg_num2", "2"])

### Second step and its wires

The second step in the pipeline receives its inputs from the previous step, and emits its outputs to the next step. Thus the ModuleStep here needs a different kind of ModuleVersion, one that has both inputs and outputs defined for. We have defined such ModuleVersion, and moreover, defined it to be the default version of our Module. This allows us to provide to the ModuleStep the Module object, which would resolve to that default ModuleVersion when needed.

### Wires

The wiring to the previous step relies on the PipelineData objects we defined before, and for them we create a new dictionary mapping to the ModuleVersion. The wiring to the next step requires us to define another pair of PipelineData objects, for which also a dictionary mapping is needed.

In [ ]:
middle_step_input_wiring = {"in1":first_sum, "in2":first_prod}
middle_sum = PipelineData("middle_sum", datastore=datastore, output_mode="mount",is_directory=False)
middle_prod = PipelineData("middle_prod", datastore=datastore, output_mode="mount",is_directory=False)
middle_step_output_wiring = {"out_sum":middle_sum, "out_prod":middle_prod}

### Middle ModuleStep - resolving to the default ModuleVersion

In [ ]:
middle_step = ModuleStep(module=module,
                         inputs_map= middle_step_input_wiring, 
                         outputs_map= middle_step_output_wiring,
                         runconfig=RunConfiguration(), compute_target=aml_compute,
                         arguments = ["--file_num1", first_sum, "--file_num2", first_prod,
                                      "--output_sum", middle_sum, "--output_product", middle_prod])

## End step and its wires

The last step in the pipeline also has input and outputs, thus its configuration would be similar to the previous step. In this case we would still use Pipeline data as the step's outputs, even though they are not read by any following step, but rather act as the end result of the pipeline.

#### Wires

In [ ]:
last_step_input_wiring = {"in1":middle_sum, "in2":middle_prod}
end_sum = PipelineData("end_sum", datastore=datastore, output_mode="mount",is_directory=False)
end_prod = PipelineData("end_prod", datastore=datastore, output_mode="mount",is_directory=False)
last_step_output_wiring = {"out_sum":end_sum, "out_prod":end_prod}

### Last ModuleStep - specifing the exact version

In [ ]:
end_step = ModuleStep(module=module, version="2",
                      inputs_map= last_step_input_wiring,
                      outputs_map= last_step_output_wiring,
                      runconfig=RunConfiguration(), compute_target=aml_compute,
                      arguments=["--file_num1", middle_sum, "--file_num2", middle_prod,
                                 "--output_sum", end_sum, "--output_product", end_prod])

## Pipeline, experiment, submission

The last thing to be done is to create a pipeline out of the previously defined steps, then create an experiment and submit the pipeline to the experiment.

In [ ]:
pipeline = Pipeline(workspace=workspace, steps=[first_step, middle_step, end_step])

In [ ]:
experiment = Experiment(workspace, 'testmodulestesp')
experiment.submit(pipeline)